In [ ]:
import re
import time
from config import username, password
from splinter import Browser
from Models.DB import DB

In [ ]:
db = DB()
db.connect()
db.createTableReviews("indeed")

In [1]:
#db.dropTables("indeed_reviews")

In [ ]:
urls = ["https://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com%2Fcmp%2FDaugherty-Business-Solutions%2Freviews&tmpl=desktop&service=&from=gnav-util-acme--acme-webapp",
        "https://www.indeed.com/cmp/Daugherty-Business-Solutions/reviews"]

In [ ]:
def login(url=urls[0]):
    pass

In [ ]:
def visitURL(url=urls[1]):
    global browser
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.driver.set_window_size(1280, 720)
    
    browser.visit(url)

In [ ]:
visitURL(url=urls[1])

In [ ]:
def scrapeReviews():
    """
        header, body, rating, role, date, response, source
    """
    
    _id = 0
    
    while browser.is_element_present_by_css("a[class='pagination__ArrowStyle__nextArrow  ']"):
        
        reviews = browser.find_by_css("div[class='hreview']")
        button = browser.find_by_css("a[class='pagination__ArrowStyle__nextArrow  ']")
        
        for review in reviews:

            _id += 1
            
            header = review.find_by_css("a[class='reviewLink']").text.replace("%", "%%").replace("$", "\$")
            body = review.find_by_css("div[class='col-sm-11 pl-sm-lg mx-0']").text.replace("%", "%%").replace("$", "\$")
            rating = float(review.find_by_css('span[class="value-title"]')._element.get_attribute('title'))
            role = review.find_by_css('span[class="authorJobTitle middle reviewer"]').text.replace("%", "%%").replace("$", "\$")
            
            try:
                date = review.find_by_css("time[class='date subtle small']").text
                
            except:
                date = "0"
                
            source = "indeed"
                
            response = "No response"
                
            
            print(f"Id: {_id}\nDate: {date}\n{header}\n{rating}\n{body}\n{source}\nResponse: {response}\n-----------------------------------------------------------------------------")
            
            db.intoReviews("indeed", header, body, rating, role, date, response, source)
            
        #simulates clicking "next" button
        browser.visit( button._element.get_attribute("href"))

In [ ]:
scrapeReviews()